In [5]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
import pandas as pd

data = files.upload()

Saving pedulilindungi.csv to pedulilindungi.csv


In [6]:
data = pd.read_csv('pedulilindungi.csv')
data.head()

,Review,Value
0,Tadi saya pakai di pondok indah mall 2 di base...,POSITIF
1,"Maaf mau nanya, anak saya unduh peduli lindung...",NEGATIF
2,Login lewat web pun gak bisa,NEGATIF
3,Mau login Tidak bisa masuk.,NEGATIF
4,Setelah ukuran font tulisanya dipeekecil akhir...,POSITIF


Casefolding

In [7]:
import re
def casefolding(Review):
  Review = Review.lower()
  Review = Review.strip(" ")
  Review = re.sub(r'[?|$|.|!2_:")(-+,]','', Review)
  return Review
data['Review'] = data['Review'].apply(casefolding)
data.head(100)

,Review,Value
0,tadi saya pakai di pondok indah mall di basem...,POSITIF
1,maaf mau nanya anak saya unduh peduli lindungi...,NEGATIF
2,login lewat web pun gak bisa,NEGATIF
3,mau login tidak bisa masuk,NEGATIF
4,setelah ukuran font tulisanya dipeekecil akhir...,POSITIF
...,...,...
95,lebih leluasa bergerak,POSITIF
96,layanan sangat cepat respon,POSITIF
97,mantap 🙏,POSITIF
98,aplikasi ga bener sering minta upgrade & sanga...,NEGATIF


Tokenizing

In [10]:
def token(Review):
  nstr = Review.split(' ')
  dat = []
  a = -1
  for hu in nstr:
    a = a + 1
  if hu == '':
    dat.append(a)
    p = 0
    b = 0
  for q in dat:
      b = q - p
      del nstr[b]
      p = p + 1
  return nstr
data['Review'] = data['Review'].apply(token)
data.head(10)

,Review,Value
0,"[tadi, saya, pakai, di, pondok, indah, mall, ,...",POSITIF
1,"[maaf, mau, nanya, anak, saya, unduh, peduli, ...",NEGATIF
2,"[login, lewat, web, pun, gak, bisa]",NEGATIF
3,"[mau, login, tidak, bisa, masuk]",NEGATIF
4,"[setelah, ukuran, font, tulisanya, dipeekecil,...",POSITIF
5,"[maaf, apakah, aplikasi, ini, bisa, berjalan, ...",NEGATIF
6,"[tidak, ada, terusan, saya, setuju, dibawah, na]",POSITIF
7,"[bz, bm, nin, yev, e7, xl, jyf, b, s, k, , i, ...",NEGATIF
8,"[ko, dah, lebih, 1, bln, sertifasi, vaksin, ke...",POSITIF
9,"[sy, kasih, 3, krn, sertifikat, vaksin, sy, ga...",NEGATIF


Filtering

In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(Review):
  filtering = stopwords.words('indonesian','english')
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, Review)
  for x in fit:
    data.append(x)
    return data
data['Review'] = data['Review'].apply(stopword_removal)
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Review,Value
0,[pakai],POSITIF
1,[maaf],NEGATIF
2,[login],NEGATIF
3,[login],NEGATIF
4,[ukuran],POSITIF


Stemming

In [42]:
! pip install Sastrawi
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(Review):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in Review:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean=[]
  d_clean= "".join(do)
  print(d_clean)
  return d_clean
data['Review'] = data['Review'].apply(stemming)
data_clean.head()

pakai
maaf
login
login
ukur
maaf
terus
bz
ko
sy
gaguna
terima
alhamdulillah
muas
google
yg
apk
sya
buka
mantap
susah
aplikasi
aplikasi
oke
gimana
aplikasi
keluh
takut
susah
mudah
bagus
heloooww
ko
input
update
hallo
login
aplikasi
log
instal
tgl
we
dngan
ambil
aplikasi
tolong
apk
kualitas
kecewa
aplikasi
gabisa
assalamualaikum
knp
responsif
aplikasi
udh
masak
bagus
ngak
gua
aplikasi
ya
log
apk
manfaat
amatiraplikasi


TypeError: ignored

TF -IDF

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data['Review'].astype('U'))
text_tf


<5259x1171 sparse matrix of type '<class 'numpy.float64'>'
	with 5148 stored elements in Compressed Sparse Row format>

Splitting Data

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_tf, data['Value'], test_size=0.2, random_state = 42)

Naive Bayes

In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)
print("MultinomialNB Accuracy:" , accuracy_score(y_test,predicted))
print("MultinomialNB Precision:" , precision_score(y_test, predicted, average="binary", pos_label="NEGATIF"))
print("MultinomialNB Recall:" , recall_score(y_test, predicted, average="binary", pos_label="NEGATIF"))
print("MultinomialNB f1_score:" , f1_score(y_test, predicted, average="binary", pos_label="NEGATIF"))

print(f'coinfusion matrix;:/n {confusion_matrix(y_test, predicted)}')
print('===============================================\n')
print(classification_report(y_test, predicted, zero_division=0))

MultinomialNB Accuracy: 0.7889733840304183
MultinomialNB Precision: 0.7702020202020202
MultinomialNB Recall: 0.9384615384615385
MultinomialNB f1_score: 0.8460471567267684
coinfusion matrix;:/n [[610  40]
 [182 220]]

              precision    recall  f1-score   support

     NEGATIF       0.77      0.94      0.85       650
     POSITIF       0.85      0.55      0.66       402

    accuracy                           0.79      1052
   macro avg       0.81      0.74      0.76      1052
weighted avg       0.80      0.79      0.78      1052

